In [62]:
import scanpy as sc
import pandas as pd 
import numpy as np
import sys
import matplotlib.pyplot as plt
import celltypist
import gc
import anndata
from celltypist import models
import h5py
import scipy.sparse as scs
from multiprocessing import Pool
import os
models.download_models()

📂 Storing models in /home/workspace/.celltypist/data/models
⏩ Skipping [1/54]: Immune_All_Low.pkl (file exists)
⏩ Skipping [2/54]: Immune_All_High.pkl (file exists)
⏩ Skipping [3/54]: Adult_COVID19_PBMC.pkl (file exists)
⏩ Skipping [4/54]: Adult_CynomolgusMacaque_Hippocampus.pkl (file exists)
⏩ Skipping [5/54]: Adult_Human_MTG.pkl (file exists)
⏩ Skipping [6/54]: Adult_Human_PancreaticIslet.pkl (file exists)
⏩ Skipping [7/54]: Adult_Human_PrefrontalCortex.pkl (file exists)
⏩ Skipping [8/54]: Adult_Human_Skin.pkl (file exists)
⏩ Skipping [9/54]: Adult_Human_Vascular.pkl (file exists)
⏩ Skipping [10/54]: Adult_Mouse_Gut.pkl (file exists)
⏩ Skipping [11/54]: Adult_Mouse_OlfactoryBulb.pkl (file exists)
⏩ Skipping [12/54]: Adult_Pig_Hippocampus.pkl (file exists)
⏩ Skipping [13/54]: Adult_RhesusMacaque_Hippocampus.pkl (file exists)
⏩ Skipping [14/54]: Autopsy_COVID19_Lung.pkl (file exists)
⏩ Skipping [15/54]: COVID19_HumanChallenge_Blood.pkl (file exists)
⏩ Skipping [16/54]: COVID19_Immune_L

In [63]:
pbmc=sc.read_h5ad('adata.h5ad')

In [64]:
pbmc.raw=pbmc

In [65]:
sc.pp.normalize_total(pbmc, target_sum=1e4)
sc.pp.log1p(pbmc)

# Annotations
levels = ['L1', 'L2', 'L3', 'L3.5']
models = {'L1': '/home//workspace/private/bri_figure_all_files_test/jupyter/BRI_Figures_Final_V2/Dataset/Celltypist_Models/ref_pbmc_clean_celltypist_model_AIFI_L1_2024-04-18.pkl',
          'L2': '/home//workspace/private/bri_figure_all_files_test/jupyter/BRI_Figures_Final_V2/Dataset/Celltypist_Models/ref_pbmc_clean_celltypist_model_AIFI_L2_2024-04-19.pkl',
          'L3': '/home//workspace/private/bri_figure_all_files_test/jupyter/BRI_Figures_Final_V2/Dataset/Celltypist_Models/ref_pbmc_clean_celltypist_model_AIFI_L3_2024-04-19.pkl'}
predictions = {}

for level, model_type in models.items():
    model_file = models[level]
    predictions[level] = celltypist.annotate(pbmc, model=model_file)

🔬 Input data has 952 cells and 58051 genes
🔗 Matching reference genes in the model
🧬 1059 features used for prediction
⚖️ Scaling input data
🖋️ Predicting labels
✅ Prediction done!
🔬 Input data has 952 cells and 58051 genes
🔗 Matching reference genes in the model
🧬 1835 features used for prediction
⚖️ Scaling input data
🖋️ Predicting labels
✅ Prediction done!
🔬 Input data has 952 cells and 58051 genes
🔗 Matching reference genes in the model
🧬 2380 features used for prediction
⚖️ Scaling input data
🖋️ Predicting labels
✅ Prediction done!


In [66]:
pbmc.obs['AIFI_L1']=predictions['L1'].predicted_labels.reset_index()['predicted_labels'].tolist()
pbmc.obs['AIFI_L2']=predictions['L2'].predicted_labels.reset_index()['predicted_labels'].tolist()
pbmc.obs['AIFI_L3']=predictions['L3'].predicted_labels.reset_index()['predicted_labels'].tolist()

In [67]:
split_columns = pbmc.obs.index.str.split('_', expand=True).to_frame(index=False)
split_columns.columns=['donor', 'days', 'age_group', 'cDNA', 'well']


In [68]:
# Merge without caring about matching indices
result = pd.concat([pbmc.obs.reset_index(drop=True), split_columns.reset_index(drop=True)], axis=1)
result.index = pbmc.obs.index

In [69]:
pbmc.obs=result

In [70]:
pbmc=pbmc.raw.to_adata()

In [72]:
pbmc.write_h5ad("adata_annoatated.h5ad")

... storing 'AIFI_L1' as categorical
... storing 'AIFI_L2' as categorical
... storing 'AIFI_L3' as categorical
... storing 'donor' as categorical
... storing 'days' as categorical
... storing 'age_group' as categorical
... storing 'cDNA' as categorical
... storing 'well' as categorical
